In [4]:
import os
import cv2
import numpy as np
import csv
!pip install ultralytics
!pip install easyocr


from ultralytics import YOLO
import easyocr

# Load the YOLO model
model = YOLO('yolov8n.pt')  # Update the model path if necessary

# Define image folder and output CSV path
image_folder = '/content/Images_Dataset/'  # Adjust this path for your Colab environment
csv_output_path = os.path.join(image_folder, 'detection_results.csv')

# Ensure the image folder exists
if not os.path.exists(image_folder):
    print(f"Image folder '{image_folder}' does not exist.")
    exit()


image_folder = '/content/Images_Dataset/'
os.makedirs(image_folder, exist_ok=True)
print(f"Image folder created at: {image_folder}")

from google.colab import files
import os

image_folder = '/content/Images_Dataset/'

# Upload files
uploaded = files.upload()  # Opens a file chooser dialog

# Save the uploaded files to the folder
for file_name in uploaded.keys():
    file_path = os.path.join(image_folder, file_name)
    with open(file_path, 'wb') as f:
        f.write(uploaded[file_name])
print(f"Uploaded images are stored in: {image_folder}")
image_files = os.listdir(image_folder)
if image_files:
    print("Uploaded files:")
    print(image_files)
else:
    print("No files found in the folder!")

# Get list of image files
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('jpg', 'jpeg', 'png'))]

if not image_files:
    print(f"No images found in folder: {image_folder}")
    exit()

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Open CSV file for writing
with open(csv_output_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Image Name", "Number of Vehicles", "Detected Plates", "Confidence Scores"])

    # Process each image
    for image_name in image_files:
        image_path = os.path.join(image_folder, image_name)
        print(f"Processing {image_path}")

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load {image_path}")
            continue

        # Perform YOLO detection
        results = model(image)
        result = results[0]

        # Vehicle classes to detect
        vehicle_classes = ["car", "truck", "bus", "motorcycle"]

        detected_vehicles = []
        detected_plates = []
        confidence_scores = []

        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = float(box.conf[0])
            class_id = int(box.cls[0])
            class_name = model.names[class_id]

            # Check if detected object is a vehicle
            if class_name in vehicle_classes:
                detected_vehicles.append(class_name)
                confidence_scores.append(f"{confidence:.2f}")

                # Crop the region of interest (ROI) for ANPR/EasyOCR
                roi = image[y1:y2, x1:x2]

                # Use EasyOCR for number plate recognition
                ocr_results = reader.readtext(roi)
                for (bbox, text, conf) in ocr_results:
                    detected_plates.append(text)

                # Draw bounding box and label on the image
                label = f"{class_name} ({confidence:.2f})"
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Write results to CSV
        csv_writer.writerow([
            image_name,
            len(detected_vehicles),
            ", ".join(detected_plates),
            ", ".join(confidence_scores)
        ])

        # Save processed image
        output_path = os.path.join(image_folder, f"output_{image_name}")
        cv2.imwrite(output_path, image)
        print(f"Processed image saved at: {output_path}")

print(f"Detection results saved to: {csv_output_path}")


Image folder created at: /content/Images_Dataset/


Saving ambassador-classic-cars-parked-outside-a-south-indian-railway-station-FGKMWK.jpg to ambassador-classic-cars-parked-outside-a-south-indian-railway-station-FGKMWK.jpg
Uploaded images are stored in: /content/Images_Dataset/
Uploaded files:
['1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident- (1).jpg', 'output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident- (1).jpg', '1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident-.jpg', 'output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident-.jpg', 'ambassador-classic-cars-parked-outside-a-south-indian-railway-station-FGKMWK.jpg', 'detection_results.csv', 'output_output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident-.jpg']
Processing /content/Images_Dataset/1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident- (1).jpg

0: 448x640 1 person, 1 car, 2 motorcycles, 1 bus, 1 truck, 297.7ms
Speed: 6.9ms preprocess, 297.7ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)
Processed image saved at: /content/Images_Dataset/outp

In [5]:
import os

image_folder = '/content/Images_Dataset/'
files = os.listdir(image_folder)
print("Files in the folder:")
print(files)


Files in the folder:
['1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident- (1).jpg', 'output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident- (1).jpg', 'output_output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident- (1).jpg', '1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident-.jpg', 'output_output_output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident-.jpg', 'output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident-.jpg', 'ambassador-classic-cars-parked-outside-a-south-indian-railway-station-FGKMWK.jpg', 'output_ambassador-classic-cars-parked-outside-a-south-indian-railway-station-FGKMWK.jpg', 'detection_results.csv', 'output_output_1da2f1b0-9a52-42a3-bac4-26300024df07_Road-accident-.jpg']


In [6]:
from google.colab import files
csv_output_path = '/content/Images_Dataset/detection_results.csv'
files.download(csv_output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>